Primero corremos un script para el build de las librerias de c++ como modulos de python.

In [ ]:
!sh build.sh

Importamos librerias necesarias

In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import metnum
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_openml
import time

Obtenemos el dataset de MNIST que contiene 70000 imagenes, partido en 4/5 de entrenamiento y 1/5 de validacion. El mismo esta shuffleado sobre si mismo.

In [ ]:
X_train, y_train, X_val, y_val = get_MNIST(0.8)

print(f"Ahora tengo {len(X_train)} instancias de entrenamiento y {len(X_val)} de validación")

Ahora buscamos tenemos como objetivo buscar el mejor par de alpha (PCA) y k (kNN). Para esto tomamos los mejores alphas que encontramos en relación a su accuracy y performance y para cada ejecutamos el algoritmo con distintos valores de k

In [ ]:
acc_heatmap = np.zeros((8, 13))
duration_heatmap = np.zeros((8, 13))
for i, alpha in enumerate([16,17,19,20,21,22,23,24]):
    print(alpha)
    for j, k in enumerate([1,2,3,4,5,6,7,8,9,10,100,200,1000]):
        print(k)
        start =  time.process_time()
        pca = metnum.PCA(alpha)
        pca.fit(X_train)
        X_train_PCA = pca.transform(X_train)

        X_val_PCA = pca.transform(X_val)

        clf_metnum = metnum.KNNClassifier(k)
        clf_metnum.fit(X_train_PCA, y_train)
        clf_mentum_predicted = clf_metnum.predict(X_val_PCA)
        end = time.process_time()
        time_performance = end - start
        acc = accuracy_score(clf_mentum_predicted, y_val)
        acc_heatmap[i, j] = acc
        duration_heatmap[i, j] = time_performance

In [ ]:
acc_heatmap

In [ ]:
duration_heatmap

Una vez que tenemos nuestras matrices de accuracy y performance cuyas celdas representan la combinacion de un k con un alpha, graficamos los resultados en un heatmap. Primero, la accuracy:

In [ ]:
df = pd.DataFrame(acc_heatmap, columns=[1,2,3,4,5,6,7,8,9,10,100,200,1000], index=[16,17,19,20,21,22,23,24])

ax = sns.heatmap(df, vmin=0.85, vmax=1, annot=True, fmt='.3f')
ax.set_xlabel("k")
ax.set_ylabel("alpha")
plt.title('Accuracy para distintas combinaciones de k y alpha')
plt.show()

Luego, la relación (ratio) entre accuracy y performance, tambien para cada combinación entre k y alpha:

In [ ]:
df = pd.DataFrame(acc_heatmap / duration_heatmap, columns=[1,2,3,4,5,6,7,8,9,10,100,200,1000], index=[16,17,19,20,21,22,23,24])

ax = sns.heatmap(df, annot=True, fmt='.3f')
ax.set_xlabel("k")
ax.set_ylabel("alpha")
plt.title('Ratio Accuracy/Duration para distintas combinaciones de k y alpha')
plt.show()